<a href="https://colab.research.google.com/github/0BVer/sch_AI/blob/main/sentiment_trian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd

In [7]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/Reviews.csv")
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
review_df1 = df[['Text', 'sentiment']]

In [26]:
print(review_df1)

                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
568449  Great for sesame chicken..this is a good if no...  positive
568450  I'm disappointed with the flavor. The chocolat...  negative
568451  These stars are small, so you can give 10-15 o...  positive
568452  These are the BEST treats for training and rew...  positive
568453  I am very satisfied ,product is as advertised,...  positive

[525814 rows x 2 columns]


In [22]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/Tweets.csv")
review_df2 = df[['text', 'airline_sentiment']]
review_df2 = review_df2[review_df2['airline_sentiment'] != 'neutral']
review_df2.columns = ["Text", "sentiment"]

In [27]:
print(review_df2)

                                                    Text sentiment
1      @VirginAmerica plus you've added commercials t...  positive
3      @VirginAmerica it's really aggressive to blast...  negative
4      @VirginAmerica and it's a really big bad thing...  negative
5      @VirginAmerica seriously would pay $30 a fligh...  negative
6      @VirginAmerica yes, nearly every time I fly VX...  positive
...                                                  ...       ...
14633  @AmericanAir my flight was Cancelled Flightled...  negative
14634         @AmericanAir right on cue with the delays👌  negative
14635  @AmericanAir thank you we got on a different f...  positive
14636  @AmericanAir leaving over 20 minutes Late Flig...  negative
14638  @AmericanAir you have my money, you change my ...  negative

[11541 rows x 2 columns]


In [23]:
review_df = pd.concat([review_df1, review_df2], ignore_index = True)
sentiment_label = review_df.sentiment.factorize()
test_sets = review_df.Text.values

In [24]:
print(review_df)

                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
537350  @AmericanAir my flight was Cancelled Flightled...  negative
537351         @AmericanAir right on cue with the delays👌  negative
537352  @AmericanAir thank you we got on a different f...  positive
537353  @AmericanAir leaving over 20 minutes Late Flig...  negative
537354  @AmericanAir you have my money, you change my ...  negative

[537355 rows x 2 columns]


In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(test_sets)

In [29]:
encoded_docs = tokenizer.texts_to_sequences(test_sets)

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen = 200)
print(padded_sequence)

[[    0     0     0 ...   101    58   141]
 [    0     0     0 ...    38    24  5778]
 [    0     0     0 ...     5     1 13888]
 ...
 [    0     0     0 ...  1161     5  4306]
 [    0     0     0 ...   751   753 19091]
 [    0     0     0 ...    96    13  8104]]


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size=len(tokenizer.word_index)+1
embeding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embeding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           4231168   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,247,819
Trainable params: 4,247,819
Non-trainable params: 0
______________________________________________

In [32]:
history = model.fit(padded_sequence, sentiment_label[0], validation_split=0.2, epochs=1, batch_size=512)

840/840 [==============================] - 1459s 2s/step - loss: 0.2302 - accuracy: 0.9112 - val_loss: 0.2680 - val_accuracy: 0.8879


In [33]:
epochs=6
batch_size = 64
history = model.fit(padded_sequence, sentiment_label[0], validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+'batch_size='+str(batch_size)+'h5')

Epoch 1/6
6717/6717 [==============================] - 2361s 352ms/step - loss: 0.1689 - accuracy: 0.9356 - val_loss: 0.2445 - val_accuracy: 0.8999
Epoch 2/6
6717/6717 [==============================] - 2440s 363ms/step - loss: 0.1366 - accuracy: 0.9486 - val_loss: 0.2640 - val_accuracy: 0.9046
Epoch 3/6
6717/6717 [==============================] - 2550s 380ms/step - loss: 0.1183 - accuracy: 0.9561 - val_loss: 0.2737 - val_accuracy: 0.9040
Epoch 4/6
6717/6717 [==============================] - 2521s 375ms/step - loss: 0.1073 - accuracy: 0.9606 - val_loss: 0.2944 - val_accuracy: 0.9027
Epoch 5/6
6717/6717 [==============================] - 2517s 375ms/step - loss: 0.0992 - accuracy: 0.9638 - val_loss: 0.2585 - val_accuracy: 0.9112
Epoch 6/6
6717/6717 [==============================] - 2524s 376ms/step - loss: 0.0935 - accuracy: 0.9661 - val_loss: 0.2675 - val_accuracy: 0.9133
INFO:tensorflow:Assets written to: trained_model(epoch=6batch_size=64h5/assets
